In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib_venn import venn2
import plotly.express as px
import plotly.graph_objects as go
import os
from datetime import datetime



In [2]:
compstak_df = pd.read_csv(r'C:\Users\clint\Desktop\compstak-analysis\Data\compstak_mapped.csv')
doe_df = pd.read_csv(r'C:\Users\clint\Desktop\compstak-analysis\Data\DOE_mapped.csv')

In [3]:
compstak_df

,Property Type,Property Subtype,Property Id,State,DOE_Compliant_Property_Type
0,Retail,Parking,1,NY,Other
1,Retail,Apartments,2,NY,Multi-Family
2,Office,NaN,3,NY,Office
3,Retail,NaN,4,NY,Retail
4,Office,Mixed-Use,5,NY,Other
...,...,...,...,...,...
759618,NaN,NaN,3611385,NY,Other
759619,Industrial,Flex/R&D,3611389,FL,Industrial
759620,Industrial,Flex/R&D,3611390,FL,Industrial
759621,NaN,NaN,3611391,CA,Other


In [4]:
doe_df

,statecode,reported_propertytype,reported_propertysubtype,compstak_equivalent_category
0,CT,Flex,Light Manufacturing,Industrial
1,CT,Industrial,Warehouse,Industrial
2,CT,Industrial,NaN,Industrial
3,CT,Multi-Family,Apartments,Multi-Family
4,CT,Multi-Family,Apartments,Multi-Family
...,...,...,...,...
2246480,WI,Retail,Storefront Retail/Office,Retail
2246481,WI,Retail,Storefront Retail/Office,Retail
2246482,WI,Retail,NaN,Retail
2246483,WI,Retail,NaN,Retail


In [5]:
# Filter the DataFrame to show only Land and Mixed-Use properties
land_mixed_rows = compstak_df[compstak_df['DOE_Compliant_Property_Type'].isin(['Land', 'Mixed-Use'])]

# Display the filtered rows
print(f"Total rows found: {len(land_mixed_rows)}")
display(land_mixed_rows)

# If you want to see counts by type
type_counts = land_mixed_rows['DOE_Compliant_Property_Type'].value_counts()
print("\nCounts by DOE_Compliant_Property_Type:")
display(type_counts)

Total rows found: 0


,Property Type,Property Subtype,Property Id,State,DOE_Compliant_Property_Type



Counts by DOE_Compliant_Property_Type:


Series([], Name: count, dtype: int64)

In [6]:
# Drop rows with 'Land' and 'Mixed-Use' DOE-compliant property types from CompStak dataset
# Create a filtered version of the CompStak dataframe

# Calculate how many rows were removed
num_rows_removed = len(compstak_df) - len(filtered_compstak_df)
percent_removed = (num_rows_removed / len(compstak_df)) * 100

# Print statistics about the filtering
print(f"Original CompStak dataset: {len(compstak_df):,} rows")
print(f"Filtered CompStak dataset: {len(filtered_compstak_df):,} rows")
print(f"Removed {num_rows_removed:,} rows ({percent_removed:.2f}% of original dataset)")

# Display the distribution of property types after filtering
filtered_counts = filtered_compstak_df['DOE_Compliant_Property_Type'].value_counts().reset_index()
filtered_counts.columns = ['Property Type', 'Count']
print("\nDOE-Compliant Property Type distribution after filtering:")
print(filtered_counts)

print("\nCompStak dataframe has been updated to the filtered version")

NameError: name 'filtered_compstak_df' is not defined

In [ ]:
compstak_df

,Property Type,Property Subtype,Property Id,State,DOE_Compliant_Property_Type
0,Retail,Parking,1,NY,Other
1,Retail,Apartments,2,NY,Multi-Family
2,Office,NaN,3,NY,Office
3,Retail,NaN,4,NY,Retail
4,Office,Mixed-Use,5,NY,Other
...,...,...,...,...,...
759618,NaN,NaN,3611385,NY,Other
759619,Industrial,Flex/R&D,3611389,FL,Industrial
759620,Industrial,Flex/R&D,3611390,FL,Industrial
759621,NaN,NaN,3611391,CA,Other


In [ ]:
unique_categories = compstak_df['DOE_Compliant_Property_Type'].unique()
print(unique_categories)

['Other' 'Multi-Family' 'Office' 'Retail' 'Industrial' 'Hotel']


In [ ]:
doe_df['compstak_equivalent_category'].unique()

array(['Industrial', 'Multi-Family', 'Office', 'Retail', 'Other', 'Hotel'],
      dtype=object)

# Creating a Sunburst Diagram of DOE and CompStak Data

This sunburst diagram visualizes the DOE dataset as the total, with the first level showing the DOE categories and the second level showing corresponding CompStak property types.

## Fixed Sunburst Visualization

Let's create a more reliable sunburst visualization that properly shows the relationship between DOE and CompStak property types.

In [ ]:
# Get counts from both datasets
doe_counts = doe_df['compstak_equivalent_category'].value_counts().reset_index()
doe_counts.columns = ['Category', 'DOE_Count']

compstak_counts = compstak_df['DOE_Compliant_Property_Type'].value_counts().reset_index()
compstak_counts.columns = ['Category', 'CompStak_Count']

# Create a simplified data structure for the sunburst
sunburst_data = []

# Root node
sunburst_data.append({
    "id": "root",
    "parent": "",
    "value": 1,  # Just a placeholder
    "label": "Property Types"
})

# Add DOE as a first-level node
sunburst_data.append({
    "id": "DOE",
    "parent": "root",
    "value": len(doe_df),
    "label": f"DOE ({len(doe_df):,})"
})

# Add CompStak as a first-level node
sunburst_data.append({
    "id": "CompStak",
    "parent": "root",
    "value": len(compstak_df),
    "label": f"CompStak ({len(compstak_df):,})"
})

# Add DOE categories
for _, row in doe_counts.iterrows():
    category = row['Category']
    count = row['DOE_Count']
    sunburst_data.append({
        "id": f"DOE_{category}",
        "parent": "DOE",
        "value": count,
        "label": f"{category} ({count:,})"
    })

# Add CompStak categories
for _, row in compstak_counts.iterrows():
    category = row['Category']
    count = row['CompStak_Count']
    sunburst_data.append({
        "id": f"CompStak_{category}",
        "parent": "CompStak",
        "value": count,
        "label": f"{category} ({count:,})"
    })

# Convert to DataFrame for the plot
sunburst_df = pd.DataFrame(sunburst_data)
sunburst_df

,id,parent,value,label
0,root,,1,Property Types
1,DOE,root,1513487,"DOE (1,513,487)"
2,CompStak,root,759623,"CompStak (759,623)"
3,DOE_Retail,DOE,625146,"Retail (625,146)"
4,DOE_Industrial,DOE,289287,"Industrial (289,287)"
5,DOE_Office,DOE,268883,"Office (268,883)"
6,DOE_Multi-Family,DOE,170267,"Multi-Family (170,267)"
7,DOE_Other,DOE,108770,"Other (108,770)"
8,DOE_Hotel,DOE,51134,"Hotel (51,134)"
9,CompStak_Retail,CompStak,195522,"Retail (195,522)"


In [ ]:
# Alternative visualization: Create a bar chart comparing DOE and CompStak
# Merge the counts
comparison_df = pd.merge(doe_counts, compstak_counts, on='Category', how='outer').fillna(0)
comparison_df = comparison_df.sort_values(by='DOE_Count', ascending=False)

# Create a grouped bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=comparison_df['Category'],
    y=comparison_df['DOE_Count'],
    name='DOE',
    marker_color='#1E88E5',
    text=comparison_df['DOE_Count'].apply(lambda x: f"{x:,.0f}"),
    textposition='outside'
))

fig.add_trace(go.Bar(
    x=comparison_df['Category'],
    y=comparison_df['CompStak_Count'],
    name='CompStak',
    marker_color='#FF8F00',
    text=comparison_df['CompStak_Count'].apply(lambda x: f"{x:,.0f}"),
    textposition='outside'
))

fig.update_layout(
    title='DOE vs CompStak Property Type Counts',
    xaxis_title='Property Type',
    yaxis_title='Count',
    barmode='group',
    width=900,
    height=600,
    template='plotly_white',
    yaxis=dict(
        type='log'  # Use log scale to better show the differences
    )
)

fig.show()

# Save the figure as a PNG image
fig.write_image(r'C:\Users\clint\Desktop\compstak-analysis\Images\Corrected Data\DOE_vs_CompStak.png')

## Remove 'Other' Category from Visualizations

The following cells filter out the 'Other' category from both DOE and CompStak datasets and recreate the sunburst and bar chart visualizations without it.

In [ ]:
# Filter out 'Other' from DOE and CompStak counts
doe_counts_no_other = doe_counts[doe_counts['Category'] != 'Other'].reset_index(drop=True)
compstak_counts_no_other = compstak_counts[compstak_counts['Category'] != 'Other'].reset_index(drop=True)

# Create sunburst data without 'Other'
sunburst_data_no_other = []
sunburst_data_no_other.append({
    "id": "root",
    "parent": "",
    "value": 1,
    "label": "Property Types"
})
sunburst_data_no_other.append({
    "id": "DOE",
    "parent": "root",
    "value": doe_counts_no_other['DOE_Count'].sum(),
    "label": f"DOE ({doe_counts_no_other['DOE_Count'].sum():,})"
})
sunburst_data_no_other.append({
    "id": "CompStak",
    "parent": "root",
    "value": compstak_counts_no_other['CompStak_Count'].sum(),
    "label": f"CompStak ({compstak_counts_no_other['CompStak_Count'].sum():,})"
})
for _, row in doe_counts_no_other.iterrows():
    sunburst_data_no_other.append({
        "id": f"DOE_{row['Category']}",
        "parent": "DOE",
        "value": row['DOE_Count'],
        "label": f"{row['Category']} ({row['DOE_Count']:,})"
    })
for _, row in compstak_counts_no_other.iterrows():
    sunburst_data_no_other.append({
        "id": f"CompStak_{row['Category']}",
        "parent": "CompStak",
        "value": row['CompStak_Count'],
        "label": f"{row['Category']} ({row['CompStak_Count']:,})"
    })
sunburst_df_no_other = pd.DataFrame(sunburst_data_no_other)
sunburst_df_no_other

,id,parent,value,label
0,root,,1,Property Types
1,DOE,root,1404717,"DOE (1,404,717)"
2,CompStak,root,591891,"CompStak (591,891)"
3,DOE_Retail,DOE,625146,"Retail (625,146)"
4,DOE_Industrial,DOE,289287,"Industrial (289,287)"
5,DOE_Office,DOE,268883,"Office (268,883)"
6,DOE_Multi-Family,DOE,170267,"Multi-Family (170,267)"
7,DOE_Hotel,DOE,51134,"Hotel (51,134)"
8,CompStak_Retail,CompStak,195522,"Retail (195,522)"
9,CompStak_Industrial,CompStak,183106,"Industrial (183,106)"


In [ ]:
# Bar chart without 'Other' category
comparison_df_no_other = pd.merge(doe_counts_no_other, compstak_counts_no_other, on='Category', how='outer').fillna(0)
comparison_df_no_other = comparison_df_no_other.sort_values(by='DOE_Count', ascending=False)

fig_no_other = go.Figure()
fig_no_other.add_trace(go.Bar(
    x=comparison_df_no_other['Category'],
    y=comparison_df_no_other['DOE_Count'],
    name='DOE',
    marker_color='#1E88E5',
    text=comparison_df_no_other['DOE_Count'].apply(lambda x: f"{x:,.0f}"),
    textposition='outside'
))
fig_no_other.add_trace(go.Bar(
    x=comparison_df_no_other['Category'],
    y=comparison_df_no_other['CompStak_Count'],
    name='CompStak',
    marker_color='#FF8F00',
    text=comparison_df_no_other['CompStak_Count'].apply(lambda x: f"{x:,.0f}"),
    textposition='outside'
))
fig_no_other.update_layout(
    title='DOE vs CompStak Property Type Counts (No Other)',
    xaxis_title='Property Type',
    yaxis_title='Count',
    barmode='group',
    width=900,
    height=600,
    template='plotly_white',
    yaxis=dict(type='log')
)
fig_no_other.show()

# Save the figure as a PNG image
fig_no_other.write_image(r'C:\Users\clint\Desktop\compstak-analysis\Images\Corrected Data\DOE_vs_CompStak_no_other.png')

In [ ]:
# Save all figures to the Images/Corrected Data directory

# Create a timestamp for unique filenames
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Ensure the directory exists
save_dir = r'C:\Users\clint\Desktop\compstak-analysis\Images\Corrected Data'
os.makedirs(save_dir, exist_ok=True)

# Create a sunburst chart and save it
fig_sunburst = px.sunburst(
    sunburst_df,
    ids='id',
    parents='parent',
    values='value',
    names='label',
    title='Property Type Distribution in DOE and CompStak Datasets'
)

fig_sunburst.update_layout(
    width=800,
    height=800
)

# Save the sunburst figure
fig_sunburst.write_image(os.path.join(save_dir, f'property_type_sunburst_{timestamp}.png'))
fig_sunburst.write_html(os.path.join(save_dir, f'property_type_sunburst_{timestamp}.html'))

# Create a stacked bar chart
fig_stacked = px.bar(
    treemap_df,
    x='Property Type',
    y='Count',
    color='Source',
    barmode='group',
    title='Property Type Distribution by Source',
    color_discrete_map={'DOE': '#1E88E5', 'CompStak': '#FF8F00'}
)

fig_stacked.update_layout(
    width=900,
    height=600
)

# Save the stacked bar figure
fig_stacked.write_image(os.path.join(save_dir, f'property_type_comparison_{timestamp}.png'))

print(f"All figures saved to {save_dir}")

All figures saved to C:\Users\clint\Desktop\compstak-analysis\Images\Corrected Data


In [ ]:
# Save compstak_df and doe_df to CSV files
compstak_df.to_csv(r'C:\Users\clint\Desktop\compstak-analysis\Data\compstak_mapped.csv', index=False)
doe_df.to_csv(r'C:\Users\clint\Desktop\compstak-analysis\Data\DOE_mapped.csv', index=False)